# Generating Sequences With Recurrent Neural Networks

Link: https://arxiv.org/abs/1308.0850

Authors: Alex Graves

Institution: Department of Computer Science University of Toronto

Publication: arXiv

Date: 2014

## Background Materials

- LSTM implementation explained https://apaszke.github.io/lstm-explained.html
- Understanding LSTM Networks http://colah.github.io/posts/2015-08-Understanding-LSTMs/

## What is this paper about?

How LSTM can generate sequences with long-range structure such as text and online handwriting.

## What is the motivation of this research?

To demonstrate LSTM can use its memory to generate complex sequences containig long-range structure.


## What makes this paper different from previous research?




## How this paper achieve it?




### RNN formulation

<img src="img/Generating Sequences With Recurrent Neural Networks Figure1.png" width="500">

The hidden layer activations are iterations of following equations,

$h_t^1 = \mathcal{H}(W_{\mathrm{input\times h^1}x_t} + W_{h^1\times h^1} + b_h^1)$  ($n = 1$)

$h_t^n = \mathcal{H}(W_{\mathrm{input}\times h^n x_t} + W_{h^{n-1}\times h^n} + W_{h^n\times h^n}h_{t-1}^n + b_h^n)$  ($2 \le n \le N$)

where $\mathcal{H}$ is an activation function, typically sigmoid. 

The probability of input sequence $x$ is 

$\mathrm{Pr}(x) = \prod_{t=1}^T\mathrm{Pr}(x_{t+1}|y_t)$

where $x = (x_1,...,x_T)$ is input vector sequence and $y = (y_1,...,y_T)$ is output vector sequence. 

The sequence loss function is negative logarithm of $\mathrm{Pr}$.

$\mathcal{L}(x) = -\sum_{t=1}^T\log\mathrm{Pr}(x_{t+1}|y_t)$


### Long Short-Term Memory formulation

While hidden layer function $\mathcal{H}$ of most RNN is an elementwise application of a sigmoid function, LSTM uses memory cells formulated as follows.

<img src="img/Generating Sequences With Recurrent Neural Networks Figure2.png" width="500">

$h_t = o_t\tanh(c_t)$

$o_t = \sigma(W_{xo}x_t + W_{ho}h_{t-1} + W_{co}c_t + b_o)$

$c_t = f_tc_{t-1} + i_t\tanh(W_{xc}x_t + W_{hc}h_{t-1} + b_c)$

$f_t = \sigma(W_{xf}x_t + W_{hf}h_{t-1} + W_{cf}c_{t-1} + b_f)$

$i_t = \sigma(W_{xi}x_t + W_{hi}h_{t-1} + W_{ci}c_{t-1} + b_i)$

where $\sigma$ is logistic sigmoid function.

### Text Prediction

Text data is discrete. The probability that text $x_t$ at time $t$ is $k$ is classification problem of $K$ classes if there is $K$ text classes in total.

$\mathrm{Pr}(x_{t+1} = k|y_t) = y_t^k = \frac{\exp(\hat{y}_t^k)}{\sum_{k=1}^K\exp(\hat{y}_t^k)}$

where $y_t = \mathcal{Y}(\hat{y_t})$ and $\mathcal{Y}$ is output layer function, here it is softmax function.

From the following equation,

$\mathcal{L}(x) = -\sum_{t=1}^T\log\mathrm{Pr}(x_{t+1}|y_t)$

the loss function is

$\mathcal{L}(x) = -\sum_{t=1}^T\log y_t^{x_{t+1}}$

### Text Prediction Results

In Penn Treebank experiments,

- two equivalent metrics is used
 - bits-per-character (BPC): average value of $-\log_2\mathrm{Pr}(x_{t+1}|y_t)$
 - perplexity: $2^{average-character-length\times \mathrm{BPC}}$
- word-level RNN performed better than the character- level network
- comparable to Mikolov (2012) but benefit of dynamic evaluation(*) is more pronounced in LSTM than RNN, which suggests rapidly adapting to new data.

(*) Dynamic evaluation is to allow a network to adapt its weight while is beeing evaluated on test data. It is legitimate for prediction problem so long as it only sees the test data once.


In Wikipedia experiments,

- character level RNN achieved 1.54 BPC and improved to 1.47 with maximum entropy model
 - for comparison the Hutter Prize (*) winner variant of PAQ-8 algorithm achieves 1.28 BPC. Mainstream compressors such as zip get more than 2.
- See generated samples by the prediction network included in this paper
 
(*) Hutter Prize is a challenge to compress the first 100 million bytes of the complete English Wikipedia data to as small as possible.



### Handwriting Prediction

The idea of mixture density network is to use the output to parameterise a mixture distribution.
Mixture density outputs can also be used with RNN. In this case the output distribution is conditioned on the history of inputs.

The basic RNN architecture is unchanged.

Each input vector $x_t$ consists of $(x_1, x_2)$ pair that is the pen offset from the previous input.
In addition a binary $x_3$ represents end of stroke (1 means the pen is lifted and 0 for otherwise).

$x_t \in \mathbb{R} \times \mathbb{R} \times \{0, 1\}$

A mixture of bivariate Gaussian was used to predict $x_1, x_2$ and Bernoulli distribution was used for $x_3$.

Each output vector $y_t$ is

$y_t = (e_t, \{\pi_t^j, \mu_t^j, \sigma_t^j, \rho_t^j\}_{j=1}^M)$

where $e$ is the end of stroke probability, $\mu^j$ is means, $\sigma^j$ is standard deviations, $\rho^j$ is collerations and $\pi^j$ is mixture weights for $M$ components.

The vectors $y_t$ are obtained from the network outputs $\hat{y}_t$.

$\hat{y}_t = (\hat{e}_t, \{\hat{w}_t^j, \hat{\mu}_t^j, \hat{\sigma}_t^j, \hat{\rho}_t^j\}_{j=1}^M) = b_y + \sum_{n=1}^NW_{h^ny}h_t^n$

Each parameter is activated as follows:

$e_t = \mathrm{sigmoid}(\hat{e}_t)$

$\pi_t^j = \mathrm{softmax}(\hat{\pi}_t^j)$

$\mu_t^j = \mathrm{identity}(\hat{\mu}_t^j)$

$\sigma_t^j = \exp(\hat{\sigma}_t^j)$

$\rho_t^j = \tanh(\hat{\rho}_t^j)$

The probability density $Pr(x_{t+1}|y_t)$ is

$\mathrm{Pr}(x_{t+1}|y_t) = \sum_{j=1}^M\pi_t^j\mathcal{N}(x_{t+1}|\mu_t^j, \sigma_t^j, \rho_t^j)\mathcal{B}((x_{t+1})_3;e_t)$

where 

$\mathcal{N}(x|\mu, \sigma, \rho) = \mathrm{Gaussian}(x| \mu, \sigma, \rho) = \frac{1}{2\pi\sigma_1\sigma_2\sqrt{1-\rho^2}}\exp[\frac{-Z}{2(1-\rho^2)}]$ 

and

$\mathcal{B}((x_{t+1})_3;e_t) = \mathrm{Bernoulli}((x_{t+1})_3; e_t) = e_t^{(x_{t+1})_3}(1-e_t)^{1 - (x_{t+1})_3}$

Note that $(x_{t+1})_3$ is 1 or 0.


### Handwriting Prediction Results

In IAM online handwriting database experiments,

- samples show strokes, letters and short words (e.g. "of" and "the")
- generated handwrighting samples are included in this paper

### Handwriting Synthesis

Handwriting synthesis is the generation of handwriting for a given text.
This can be done by conditioning a prediction network with annotation sequence (a character string).

This problem is difficult because

> The main challenge in conditioning the predictions on the text is that the two sequences are of very different lengths (the pen trace being on average twenty five times as long as the text), and the alignment between them is unknown until the data is generated. This is because the number of co-ordinates used to write each character varies greatly according to style, size, pen speed etc.

<img src="img/Generating Sequences With Recurrent Neural Networks Figure12.png" width="500">

The difference of the network architectire is the added input from the character sequence mediated by the window layer.

Given a length $U$ character sequence $c$ and a length $T$ data sequence $x$, the soft window $w_t$ at timestep $t$ $(1 \le t \le T)$ is defined as,

$w_t = \sum_{u=1}^U\phi(t, u)c_u$

where $\phi(t, u)$ is the window weight of $c_u$ at timestep $t$.

$\phi(t, u) = \sum_{k=1}^K\alpha_t^k\exp(-\beta_t^k(\kappa_t^k-u)^2)$

> Intuitively, the $\kappa_t$ parameters control the location of the window, the $\beta_t$ parameters control the width of the window and the $\alpha_t$ parameters control the importance of the window within the mixture.

The equation for the output layer remain unchanged.

The output probability is

$\mathrm{Pr}(x|c) = \prod_{t=1}^T\mathrm{Pr}(x_{t+1}|y_t)$

The sequence loss is 

$\mathcal{L}(x) = -\log\mathrm{Pr}(x|c)$

### Unbiased Sampling Results

- draw $x_{t+1}$ until $\phi(t, U + 1) > \phi(t, u) \forall 1 \le u \le U$
- generated samples are included in this paper

### Biased Sampling Results

Unbiased sampling generates handwritings that are not easy to read.

To obtain easy-to-read handwritings, more probable prediction is selected by biasing.
This is based on an expectation that good handwriting are more predictable than bad one.

Define the probability bias $b$ as a real number greater than or equal to zero.

Gaussian mixture is recalculated to

$\sigma_t^j = \exp(\hat{\sigma}_t^j - b)$

and each mixture weight is recalculated to

$\pi_t^j = \mathrm{softmax}(\hat{\pi}_t^j(1 + b)) = \frac{\exp(\hat{\pi}_t^j(1 + b))}{\sum_{j' = 1}^M\exp(\hat{\pi}_t^{j'}(1 + b))}$

This artificially reduces the variance.
When $b = 0$ unbiased sampling is recovered.

The generated samples are included in the paper.

### Primed Sampling Results

Primed sampling generates handwriting in the style of a particular writer.

This can be achieved by starting with a real sequence that is to be mimiced and then generating an extension.
This works because the real sequence to be mimiced still exists in the network's memory.

The generated samples are included in the paper.

## Dataset used in this study

- Penn Treebank
- Hutter Prize Wikipedia datasets
- IAM Online Handwriting Database

## Implementations

- https://sourceforge.net/projects/rnnl/
- https://github.com/szcom/rnnlib (GitHub fork)


## Further Readings

- https://www.cs.toronto.edu/~graves/gen_seq_rnn.pdf (Slides)
- http://www.cs.toronto.edu/~graves/handwriting.html (Online Demo)
- http://people.idsia.ch/~juergen/lstm/ (LSTM tutorial)
- [Gradient-based learning algorithms for recurrent networks and their computational complexity](http://dl.acm.org/citation.cfm?id=201801) (backpropagation through time explained)